In [2]:
!pip install -q openpyxl-image-loader chromadb langchain-google-genai langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 64.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 62.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 38.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 52.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.9/438.9 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.7/367.7 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.

In [3]:
from openpyxl import load_workbook
from openpyxl_image_loader import SheetImageLoader
import os

wb = load_workbook('/kaggle/input/xl-mmrag/Math-samples.xlsx')
sheet = wb.active
img_loader = SheetImageLoader(sheet)

# Print the coordinates of images found by the loader
print("Images found at coordinates:", img_loader._images.keys())

for row in range(1, sheet.max_row+1):
    cell = sheet.cell(row, 3)  # images are in column C (3rd column)
    # Check if the cell contains an image before attempting to extract it
    if cell.coordinate in img_loader._images:
        img = img_loader.get(cell.coordinate)
        # Convert RGBA to RGB before saving as JPEG
        if img.mode == 'RGBA':
            img = img.convert('RGB')
        img.save(f"image_{row}.jpg")
        print(f"Image extracted from {cell.coordinate} and saved as image_{row}.jpg")
    else:
        print(f"No image found at {cell.coordinate}")

print("Extraction process finished.")
print("Files in the current directory:", os.listdir())

Images found at coordinates: dict_keys(['C2'])
No image found at C1
Image extracted from C2 and saved as image_2.jpg
No image found at C3
No image found at C4
No image found at C5
No image found at C6
No image found at C7
No image found at C8
No image found at C9
No image found at C10
No image found at C11
No image found at C12
No image found at C13
No image found at C14
Extraction process finished.
Files in the current directory: ['.virtual_documents', 'image_2.jpg']


In [5]:

import pandas as pd
df = pd.read_excel('/kaggle/input/xl-mmrag/Math-samples.xlsx')
# C is image column
df['image_path'] = df.index.to_series().apply(lambda idx: f"image_{idx+1}.jpg" if os.path.exists(f"image_{idx+1}.jpg") else None)
df

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Q. No.,Question,Image/Figure,image_path
0,1,"In ∆ABC, right-angled at B, AB = 24 cm, BC = 7...",NaN,None
1,2,"In Fig. 8.13, find tan P – cot R.",NaN,image_2.jpg
2,3,"If sin A = 3/4, calculate cos A and tan A.",NaN,None
3,4,"Given 15 cot A = 8, find sin A and sec A.",NaN,None
4,5,"Given sec θ = 13/12, calculate all other trigo...",NaN,None
5,6,If ∠A and ∠B are acute angles such that cos A ...,NaN,None
6,7,"If cot θ = 7/8, evaluate:\n(i) (1 + sin θ)(1 –...",NaN,None
7,8,"If 3 cot A = 4, check whether (1 – tan² A)/(1 ...",NaN,None
8,9,"In triangle ABC, right-angled at B, if tan A =...",NaN,None
9,10,"In ∆PQR, right-angled at Q, PR + QR = 25 cm an...",NaN,None


In [10]:
from transformers import CLIPConfig, CLIPModel, CLIPProcessor
import torch
from PIL import Image
import numpy as np

# Load Long-CLIP
model_id = "zer0int/LongCLIP-GmP-ViT-L-14"
config = CLIPConfig.from_pretrained(model_id)
config.text_config.max_position_embeddings = 248
clip_model = CLIPModel.from_pretrained(model_id, config=config)
processor = CLIPProcessor.from_pretrained(model_id)

device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model = clip_model.to(device)

def embed_text(text):
    inputs = processor(
        text=[text],
        return_tensors="pt",
        padding="max_length",
        max_length=248
    ).to(device)
    outputs = clip_model.get_text_features(**inputs)
    # Detach gradients, move to CPU, then numpy
    return outputs.detach().cpu().numpy()[0]

def embed_image(img_path):
    img = Image.open(img_path).convert("RGB")
    inputs = processor(images=img, return_tensors="pt").to(device)
    outputs = clip_model.get_image_features(**inputs)
    return outputs.detach().cpu().numpy()[0]

embeddings = []
for idx, row in df.iterrows():
    text = row['Question']
    text_emb = embed_text(text)

    if row['image_path']:
        print(f"Row {idx}: embedding with image")
        img_emb = embed_image(row['image_path'])
        emb = (text_emb + img_emb) / 2
    else:
        print(f"Row {idx}: embedding with text only")
        emb = text_emb

    embeddings.append(emb)

Row 0: embedding with text only
Row 1: embedding with image
Row 2: embedding with text only
Row 3: embedding with text only
Row 4: embedding with text only
Row 5: embedding with text only
Row 6: embedding with text only
Row 7: embedding with text only
Row 8: embedding with text only
Row 9: embedding with text only
Row 10: embedding with text only
Row 11: embedding with text only
Row 12: embedding with text only


In [11]:
import chromadb
from chromadb.config import Settings

client = chromadb.PersistentClient(path="mm_rag_db", settings=Settings(anonymized_telemetry=False))
col = client.create_collection("excel_rows")

for idx, emb in enumerate(embeddings):
    col.add(
        embeddings=[emb.tolist()],
        metadatas=[{"row": idx}],
        ids=[str(idx)]
    )

In [12]:
# This will retrieve all stored items (with limit)
results = col.get(include=['embeddings', 'metadatas', 'documents'])

for i in range(len(results['ids'])):
    print(f"ID: {results['ids'][i]}")
    print(f"Metadata: {results['metadatas'][i]}")
    print(f"Embedding length: {len(results['embeddings'][i])}")
    print('-' * 40)

ID: 0
Metadata: {'row': 0}
Embedding length: 768
----------------------------------------
ID: 1
Metadata: {'row': 1}
Embedding length: 768
----------------------------------------
ID: 2
Metadata: {'row': 2}
Embedding length: 768
----------------------------------------
ID: 3
Metadata: {'row': 3}
Embedding length: 768
----------------------------------------
ID: 4
Metadata: {'row': 4}
Embedding length: 768
----------------------------------------
ID: 5
Metadata: {'row': 5}
Embedding length: 768
----------------------------------------
ID: 6
Metadata: {'row': 6}
Embedding length: 768
----------------------------------------
ID: 7
Metadata: {'row': 7}
Embedding length: 768
----------------------------------------
ID: 8
Metadata: {'row': 8}
Embedding length: 768
----------------------------------------
ID: 9
Metadata: {'row': 9}
Embedding length: 768
----------------------------------------
ID: 10
Metadata: {'row': 10}
Embedding length: 768
----------------------------------------
ID: 11
M